Script for logE3T estimation

In [1]:
%matplotlib inline
import sys
import random
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import glob
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from cartopy import crs, feature

sys.path.append('/ocean/vvalenzuela/MOAD/Ocean_Parcels')

from OP_functions import *

import cmocean
cmap = cmocean.cm.deep

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import statsmodels.api as sm

**Equations**

- $factor = 1 + \bigg(\frac{ssh}{td}\bigg)$


- $ln(Z) = ln(e3t * factor * 0.5)$


- $u^{*} = \frac{U \cdot k}{ln(\frac{Z}{z^{*}})}$


- $\tau = u^{*2} \cdot \rho$

In [27]:
td = np.linspace(5, 350, 300)
ssh = np.linspace(-3,3,300)
e3t = np.linspace(5,20,300)
uv_h = np.linspace(1e-3, 1, 300)
z_c = np.log(0.07)
k = 0.42

In [29]:
factor = np.zeros((len(ssh), len(td)))
ln_e3t = np.zeros((len(ssh), len(td)))

In [30]:
for i in range(0,len(td)):
    for j in range(0,len(ssh)):
        ln_e3t[i,j] = np.log(e3t[j] * (1 + (ssh[j] / td[i])) * 0.5)

In [ ]:
SSH, UV_H = np.meshgrid(ssh,uv_h)